## Goal
The main purpose of this notebook is to develope the code to read in phenotypes in desired format.
At the end, I should wrap it up as the function.

In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://nucleus.cels.anl.gov:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /vol/bmd/yanyul/GitHub/ptrs-ukb/notebook/hail-20191213-1423-0.2.28-61941242c15d.log


In [2]:
# just to load chr22 for testing purpose
mt = hl.import_bgen(
    '/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr22_v3.bgen',
    entry_fields = ['dosage'],
    index_file_map = {'/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr22_v3.bgen' : '/vol/bmd/yanyul/UKB/bgen_idx/ukb_imp_chr22_v3.bgen.idx2'},
    sample_file = '/vol/bmd/data/ukbiobank/genotypes/v3/ukb19526_imp_chr1_v3_s487395.sample')

2019-12-13 14:23:39 Hail: INFO: Number of BGEN files parsed: 1
2019-12-13 14:23:39 Hail: INFO: Number of samples in BGEN files: 487409
2019-12-13 14:23:39 Hail: INFO: Number of variants across all BGEN files: 1255683


In [3]:
mt = mt.annotate_cols(eid = mt.s.replace("\_\d+", ""))
mt = mt.key_cols_by('eid')
mt = mt.repartition(100)

In [4]:
import pandas as pd
import numpy as np

In [5]:
covar_names = 'age_recruitment,sex,pc1,pc2'
pheno_names = 'ht,mcv,mch'
indiv_id = 'eid'
int_names = 'age_recruitment,sex'
str_names = 'eid'

In [6]:
import sys 
sys.path.insert(0, '../code/')
from importlib import reload 

import my_hail_helper as myhelper

myhelper = reload(myhelper)

# Copied to ../code/my_hail_helper.py
# def names_to_list(names):
#     return names.split(',')
# def get_dtype_dic(int_names, str_names, all_names):
#     out_dic = { a : np.float for a in all_names }
#     for i in int_names:
#         out_dic[i] = np.int
#     for i in str_names:
#         out_dic[i] = str
#     return out_dic
# def read_and_split_phenotype_csv(csv_path, pheno_names, covar_names, indiv_id, int_names, str_names):
#     # read in the phenotype table prepared from ukbREST output (in CSV format)
#     # split and return two tables
#     # 1. covariate table
#     # 2. trait table
#     covar_list = names_to_list(covar_names)
#     pheno_list = names_to_list(pheno_names)
#     all_colnames = covar_list + pheno_list + names_to_list(indiv_id)
#     type_dic = get_dtype_dic(names_to_list(int_names), names_to_list(str_names), all_colnames)
#     pheno = pd.read_csv(csv_path, dtype = type_dic, usecols = list(type_dic.keys())).rename(columns = {indiv_id : 's'})
#     covar_table = pheno[covar_list + ['s']]
#     trait_table = pheno[pheno_list + ['s']]
#     return covar_table, trait_table
# def read_indiv_list(file_path):
#     # read in the file from file_path
#     # and return the first columns as a list of string 
#     indiv_list = pd.read_csv(file_path, sep = '\s+', dtype = str)
#     return indiv_list.iloc[:,0].to_list()
# def subset_by_col(df, colname, target_list):
#     return df[df[colname].isin(target_list)]


In [7]:
covar, trait = myhelper.read_and_split_phenotype_csv(
    '../output/query_phenotypes_cleaned_up.csv',
    pheno_names = pheno_names,
    covar_names = covar_names,
    indiv_id = indiv_id,
    int_names = int_names,
    str_names = str_names
)

#### Now that we've loaded in the full covariate and trait tables
Here we start to loop over all subsets and build the "list of lists" for traits.

In [8]:
subset_dic = {}
nsubset = 2
for subset_idx in range(1, nsubset + 1):
    subset_indiv_list = myhelper.read_indiv_list(f'../output/data_split/British-training-{subset_idx}.txt')
    sub_trait = myhelper.subset_by_col(trait, 'eid', subset_indiv_list)
    sub_trait = myhelper.df_to_ht(sub_trait, 'eid')  # hl.Table.from_pandas(sub_trait, key = 's')
#     sub_trait = sub_trait.repartition(40)
    subset_dic[f'subset_{subset_idx}'] = sub_trait

2019-12-13 14:23:57 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-12-13 14:24:09 Hail: INFO: Ordering unsorted dataset with network shuffle


In [9]:
covar = myhelper.df_to_ht(covar, 'eid')

2019-12-13 14:24:24 Hail: INFO: Ordering unsorted dataset with network shuffle


In [10]:
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
    'eid': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'varid': str
----------------------------------------
Entry fields:
    'dosage': float64
----------------------------------------
Column key: ['eid']
Row key: ['locus', 'alleles']
----------------------------------------


In [11]:
annot_expr_ = {
    k : subset_dic[k][mt.eid] for k in list(subset_dic.keys())
}

In [12]:
mt = mt.annotate_cols(**annot_expr_)
mt = mt.annotate_cols(covariates = covar[mt.eid])

In [13]:
# prepare trait and covar into list or list of lists
subset_list = [ [ mt[f'subset_{i}'][j] for j in mt[f'subset_{i}'] ] for i in range(1, nsubset + 1) ]
subset_names = [ [ f'subset_{i}_x_{j}' for j in mt[f'subset_{i}'] ] for i in range(1, nsubset + 1) ]
covar_list = [ mt.covariates[i] for i in list(mt.covariates.keys()) ]

In [14]:
gwas_out = hl.linear_regression_rows(
    y = subset_list,
    x = mt.dosage,
    covariates = [1] + covar_list,
    pass_through = ['varid', 'rsid']
)

2019-12-13 15:03:41 Hail: WARN: 140933 of 487409 samples have a missing phenotype or covariate.
2019-12-13 15:03:42 Hail: WARN: 140933 of 487409 samples have a missing phenotype or covariate.
2019-12-13 15:03:42 Hail: INFO: linear_regression_rows[0]: running on 346476 samples for 3 response variables y,
    with input variable x, and 5 additional covariates...
2019-12-13 15:03:42 Hail: INFO: linear_regression_rows[1]: running on 346476 samples for 3 response variables y,
    with input variable x, and 5 additional covariates...


In [15]:
gwas_out = gwas_out.annotate_globals(phenotypes = subset_names)
gwas_out.describe()

----------------------------------------
Global fields:
    'phenotypes': array<array<str>> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'varid': str 
    'rsid': str 
    'n': array<int32> 
    'sum_x': array<float64> 
    'y_transpose_x': array<array<float64>> 
    'beta': array<array<float64>> 
    'standard_error': array<array<float64>> 
    't_stat': array<array<float64>> 
    'p_value': array<array<float64>> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


In [16]:
gwas_out = gwas_out.annotate( 
    variant = hl.delimit(
        hl.array([
            gwas_out['locus'].contig,
            hl.str(gwas_out['locus'].position),
            gwas_out['alleles'][0],
            gwas_out['alleles'][1]
        ]), 
    delimiter = ':')
)
gwas_out = gwas_out.key_by('variant')
## Hey, this repartition is important
## in the sense that it avoids the unnecessary and repeated sorting caused by key_by
gwas_out = gwas_out.repartition(40)
gwas_out = gwas_out.cache()
phenotypes = gwas_out['phenotypes'].collect()[0]
for i, subset in enumerate(phenotypes):
    for j, trait in enumerate(subset):
        ht_export = myhelper.gwas_formater_from_neale_lab(gwas_out, i, j)
        ht_export.export(f'test_output/gwas_{trait}.tsv')

2019-12-13 16:53:21 Hail: INFO: Coerced sorted dataset
2019-12-13 19:24:55 Hail: INFO: merging 40 files totalling 129.7M...
2019-12-13 19:24:56 Hail: INFO: while writing:
    test_output/gwas_subset_1_x_ht.tsv
  merge time: 1.440s
2019-12-13 19:24:58 Hail: INFO: merging 40 files totalling 129.7M...
2019-12-13 19:25:00 Hail: INFO: while writing:
    test_output/gwas_subset_1_x_mcv.tsv
  merge time: 1.475s
2019-12-13 19:25:02 Hail: INFO: merging 40 files totalling 129.7M...
2019-12-13 19:25:05 Hail: INFO: while writing:
    test_output/gwas_subset_1_x_mch.tsv
  merge time: 3.062s
2019-12-13 19:25:07 Hail: INFO: merging 40 files totalling 129.7M...
2019-12-13 19:25:09 Hail: INFO: while writing:
    test_output/gwas_subset_2_x_ht.tsv
  merge time: 1.828s
2019-12-13 19:25:11 Hail: INFO: merging 40 files totalling 129.7M...
2019-12-13 19:25:14 Hail: INFO: while writing:
    test_output/gwas_subset_2_x_mcv.tsv
  merge time: 3.071s
2019-12-13 19:25:16 Hail: INFO: merging 40 files totalling 129